In [1]:
!pip install qiskit
!pip install qiskit_aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 82.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 96.2 MB/s eta 0:00:00


In [2]:
# Deutsch–Jozsa Algorithm using Qiskit 2.x
# Compatible with Qiskit >= 2.0.0

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt


In [3]:
# ---------- ORACLES ----------
def oracle_constant(qc, ancilla, value=0):
    """Constant oracle: f(x)=0 or f(x)=1"""
    if value == 1:
        qc.x(ancilla)


def oracle_balanced_parity(qc, inputs, ancilla):
    """Balanced oracle: f(x) = x0 XOR x1 XOR ... XOR xn"""
    for q in inputs:
        qc.cx(q, ancilla)




In [4]:
# ---------- DEUTSCH–JOZSA CIRCUIT ----------
def deutsch_jozsa_circuit(n, oracle_func, *oracle_args):
    """
    n: number of input qubits
    oracle_func: oracle function to modify the circuit
    oracle_args: extra arguments for oracle
    """
    qreg = QuantumRegister(n + 1, "q")
    creg = ClassicalRegister(n, "c")
    qc = QuantumCircuit(qreg, creg)

    inputs = list(range(n))
    ancilla = n

    # Step 1: Initialize |0...0>|1>
    qc.x(ancilla)

    # Step 2: Apply Hadamard to all qubits
    qc.h(qreg)

    # Step 3: Oracle
    oracle_func(qc, *oracle_args)

    # Step 4: Apply Hadamard to input qubits
    for q in inputs:
        qc.h(q)

    # Step 5: Measure only input qubits
    qc.measure(inputs, creg)

    return qc


In [10]:
# ---------- EXECUTION ----------
def run_dj(qc):
    """Run Deutsch–Jozsa circuit on AerSimulator with optional noise"""
    # Task 3: Add Noise Simulation
    from qiskit_aer.noise import NoiseModel, depolarizing_error

    # Create a simple noise model
    # This is a basic example, more realistic models can be built.
    noise_model = NoiseModel()
    # Add a depolarizing error to all single qubit gates
    error = depolarizing_error(0.05, 1) # 5% error rate
    noise_model.add_all_qubit_quantum_error(error, ['u1', 'u2', 'u3', 'x', 'y', 'z', 'h', 's', 'sdg', 't', 'tdg'])
    # Add a depolarizing error to all two qubit gates
    error_cx = depolarizing_error(0.1, 2) # 10% error rate for CX
    noise_model.add_all_qubit_quantum_error(error_cx, ['cx'])


    simulator = AerSimulator(noise_model=noise_model)
    tqc = transpile(qc, simulator)
    job = simulator.run(tqc, shots=1024)
    result = job.result()
    counts = result.get_counts()

    print("Measurement counts:", counts)
    plot_histogram(counts)
    plt.show()

    n = qc.num_clbits
    if counts.get("0" * n, 0) > 512: # With noise, we might not get 100% '0'*n
        print("✅ Function is likely CONSTANT")
    else:
        print("✅ Function is likely BALANCED")

In [9]:
# ---------- MAIN ----------
if __name__ == "__main__":
    # Test with different numbers of input qubits (Task 2)
    for n in [2, 4, 5]:
        print(f"\n=== Testing with n = {n} ===")

        print("\n--- Constant Oracle (f(x)=0) ---")
        qc_const = deutsch_jozsa_circuit(
            n, oracle_constant, n, 0
        )
        print(qc_const.draw(fold=-1))
        run_dj(qc_const)

        print("\n--- Balanced Oracle (Parity) ---")
        qc_balanced = deutsch_jozsa_circuit(
            n, oracle_balanced_parity, list(range(n)), n
        )
        print(qc_balanced.draw(fold=-1))
        run_dj(qc_balanced)

        # You can also test your custom balanced oracle here with different n
        # print(f"\n--- Custom Balanced Oracle (n={n}) ---")
        # qc_custom_balanced = deutsch_jozsa_circuit(
        #     n, oracle_custom_balanced, list(range(n)), n
        # )
        # print(qc_custom_balanced.draw(fold=-1))
        # run_dj(qc_custom_balanced)


=== Testing with n = 2 ===

--- Constant Oracle (f(x)=0) ---
     ┌───┐┌───┐┌─┐   
q_0: ┤ H ├┤ H ├┤M├───
     ├───┤├───┤└╥┘┌─┐
q_1: ┤ H ├┤ H ├─╫─┤M├
     ├───┤├───┤ ║ └╥┘
q_2: ┤ X ├┤ H ├─╫──╫─
     └───┘└───┘ ║  ║ 
c: 2/═══════════╩══╩═
                0  1 
Measurement counts: {'00': 1024}
✅ Function is CONSTANT

--- Balanced Oracle (Parity) ---
     ┌───┐          ┌───┐     ┌─┐   
q_0: ┤ H ├───────■──┤ H ├─────┤M├───
     ├───┤       │  └───┘┌───┐└╥┘┌─┐
q_1: ┤ H ├───────┼────■──┤ H ├─╫─┤M├
     ├───┤┌───┐┌─┴─┐┌─┴─┐└───┘ ║ └╥┘
q_2: ┤ X ├┤ H ├┤ X ├┤ X ├──────╫──╫─
     └───┘└───┘└───┘└───┘      ║  ║ 
c: 2/══════════════════════════╩══╩═
                               0  1 
Measurement counts: {'11': 1024}
✅ Function is BALANCED

=== Testing with n = 4 ===

--- Constant Oracle (f(x)=0) ---
     ┌───┐┌───┐┌─┐         
q_0: ┤ H ├┤ H ├┤M├─────────
     ├───┤├───┤└╥┘┌─┐      
q_1: ┤ H ├┤ H ├─╫─┤M├──────
     ├───┤├───┤ ║ └╥┘┌─┐   
q_2: ┤ H ├┤ H ├─╫──╫─┤M├───
     ├───┤├───┤ ║  ║ └╥┘┌─┐
q_3

In [7]:
# ---------- CUSTOM BALANCED ORACLE ----------
def oracle_custom_balanced(qc, inputs, ancilla):
    """Custom balanced oracle: flips ancilla for half of all inputs."""
    # Example: Flips ancilla for inputs where the first qubit is 1
    qc.cx(inputs[0], ancilla)

    # Another example: Flips ancilla for inputs 010, 101, 110, 001 for n=3
    # This requires a bit more complex logic using multi-controlled gates.
    # For n=3, inputs are q[0], q[1], q[2]. Ancilla is q[3].
    # Example for 010: ccx(0, 2, 3) will flip ancilla if q[0] and q[2] are 1.
    # We need to target specific bitstrings.
    # To target 010, we can do x(0), x(2), ccx(0, 2, 3), x(0), x(2)
    # This flips ancilla if q[0] and q[2] are 0.
    # Let's create a more general balanced oracle that targets specific states.
    # For n=3, there are 8 states (000 to 111). We need to flip for 4 states.
    # Let's choose to flip for states 001, 010, 100, 111

    # Flip for 001: x(0), x(1), ccx(0, 1, ancilla), x(0), x(1)
    qc.x(inputs[0])
    qc.x(inputs[1])
    qc.ccx(inputs[0], inputs[1], ancilla)
    qc.x(inputs[0])
    qc.x(inputs[1])

    # Flip for 010: x(0), x(2), ccx(0, 2, ancilla), x(0), x(2)
    qc.x(inputs[0])
    qc.x(inputs[2])
    qc.ccx(inputs[0], inputs[2], ancilla)
    qc.x(inputs[0])
    qc.x(inputs[2])

    # Flip for 100: x(1), x(2), ccx(1, 2, ancilla), x(1), x(2)
    qc.x(inputs[1])
    qc.x(inputs[2])
    qc.ccx(inputs[1], inputs[2], ancilla)
    qc.x(inputs[1])
    qc.x(inputs[2])

    # Flip for 111: ccx(0, 1, ancilla) followed by cx(2, ancilla) if both 0 and 1 are 1.
    # Or simply a CCX on all three inputs for n=3
    qc.ccx(inputs[0], inputs[1], ancilla)
    qc.cx(inputs[2], ancilla)

In [12]:
!pip install qiskit-ibm-runtime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.4/377.4 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.8/75.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 10.7 MB/s eta 0:00:00


### Analyzing the Oracle's Unitary

The unitary matrix of an oracle $U_f$ operating on $n$ input qubits and one ancilla qubit (initialized to $|-\rangle$) has the property:

$U_f |x\rangle |-\rangle = (-1)^{f(x)} |x\rangle |-\rangle$

where $f(x)$ is the function the oracle computes.

*   If $f(x) = 0$ (for a constant oracle), the phase is $(-1)^0 = 1$, so $U_f |x\rangle |-\rangle = |x\rangle |-\rangle$. The oracle does not change the state.
*   If $f(x) = 1$ (for a constant oracle), the phase is $(-1)^1 = -1$, so $U_f |x\rangle |-\rangle = -|x\rangle |-\rangle$. The oracle applies a phase flip.
*   If $f(x)$ is balanced, the oracle applies a phase flip $(-1)$ for exactly half of the possible input states $|x\rangle$, and no phase change $(+1)$ for the other half.

When you print the unitary matrix, you'll see a $(2^{n+1}) \times (2^{n+1})$ matrix. The effect of the oracle on the $|x\rangle |-\rangle$ state is related to the diagonal elements of this matrix, particularly how it introduces a phase of $+1$ or $-1$ for each input state $|x\rangle$.

For a diagonal oracle (which many simple oracles are, when the ancilla is in the $|-\rangle$ state), the diagonal entry corresponding to the state $|x\rangle |-\rangle$ will be $(-1)^{f(x)}$. By looking at these diagonal entries, you can infer the function $f(x)$ that the oracle implements.

In [16]:
from qiskit_ibm_runtime import QiskitRuntimeService

# Replace YOUR_TOKEN_HERE with your actual token string
QiskitRuntimeService.save_account(channel="ibm_quantum_platform", token="NW68onlia1Q2QvBomKyJNw7P4S7OwWcVk_FcOpBo5qsg")

In [31]:
# ---------- RUN ON IBM QUANTUM DEVICE ----------
# Task 4: Run on IBM Quantum Device

from qiskit_ibm_runtime import QiskitRuntimeService, Sampler
from google.colab import userdata
from qiskit import transpile
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt


# You will need to replace "__YOUR_IBM_QUANTUM_TOKEN__" with your actual token
# You can find your token in your IBM Quantum Experience account settings.
# service = QiskitRuntimeService(channel="ibm_quantum", token="__YOUR_IBM_QUANTUM_TOKEN__")

# If you have saved your token previously, you can initialize without the token argument:
# service = QiskitRuntimeService(channel="ibm_quantum")

# --- Replace the line below with one of the options above, using your token ---
# Example using token directly (LESS SECURE):
service = QiskitRuntimeService(channel="ibm_quantum_platform", token="NW68onlia1Q2QvBomKyJNw7P4S7OwWcVk_FcOpBo5qsg")

# Example using token from Colab Secrets (RECOMMENDED):
# service = QiskitRuntimeService(channel="ibm_quantum_platform", token=userdata.get('IBM_QUANTUM_TOKEN'))
# -----------------------------------------------------------------------------


# List available backends
print(service.backends())

# Choose a suitable backend (e.g., a small device like 'ibm_brisbane' or 'ibm_osaka' for testing)
backend = service.backend("ibm_fez") # Replace with an available backend

# Now you can transpile and run your circuit on the selected backend using the Sampler primitive
# We'll run the constant oracle circuit (qc_const) as an example.
# You can change this to qc_balanced or qc_custom_balanced if you prefer.

# Transpile the circuit to the backend's basis gates using a compatible translation method
transpiled_qc_const = transpile(qc_const, backend, translation_method='translator')

# Instantiate the Sampler primitive
sampler = Sampler(backend)

# Run the transpiled circuit using the Sampler
job_real = sampler.run([transpiled_qc_const], shots=1024)

print(f"Job ID: {job_real.job_id()}")
print(f"Job status: {job_real.status()}")

# You can retrieve the results later
# retrieved_job = service.job(job_real.job_id())
# real_counts = retrieved_job.result().get_counts()
# print("Real device measurement counts:", real_counts)
# plot_histogram(real_counts)
# plt.show()

qiskit_runtime_service._discover_account:WARNING:2025-11-05 16:30:46,093: Loading account with the given token. A saved account will not be used.
qiskit_runtime_service.__init__:WARNING:2025-11-05 16:30:48,082: Instance was not set at service instantiation. Free and trial plan instances will be prioritized. Based on the following filters: (tags: None, region: us-east, eu-de), and available plans: (open), the available account instances are: open-instance. If you need a specific instance set it explicitly either by using a saved account with a saved default instance or passing it in directly to QiskitRuntimeService().
qiskit_runtime_service.backends:WARNING:2025-11-05 16:30:48,083: Loading instance: open-instance, plan: open
qiskit_runtime_service.backends:WARNING:2025-11-05 16:30:48,995: Using instance: open-instance, plan: open


[<IBMBackend('ibm_fez')>, <IBMBackend('ibm_torino')>, <IBMBackend('ibm_marrakesh')>]
Job ID: d45nned63mfc73a77djg
Job status: QUEUED


In [21]:
backend = service.backend("ibm_fez")  # choose your desired device from the available ones
print(backend.configuration().basis_gates)

qiskit_runtime_service.backends:WARNING:2025-11-05 16:25:38,737: Using instance: open-instance, plan: open


['cz', 'id', 'rz', 'sx', 'x']


In [38]:
# ---------- CIRCUIT ANALYSIS ----------
# Task 5: Circuit Analysis

# Let's analyze the custom balanced oracle for n=3 as an example
n = 3
qreg = QuantumRegister(n + 1, "q")
qc_oracle_only = QuantumCircuit(qreg)

# Apply the custom balanced oracle to the circuit
# We need to initialize the ancilla in the |-> state for the oracle to work correctly
# in the Deutsch-Jozsa algorithm, but for just analyzing the unitary,
# we can apply the oracle to the |0>|0>...|0>|0> state.
# However, the definition of the oracle itself doesn't depend on the input state,
# only on the gates applied.

# We need to pass the inputs and ancilla qubits to the oracle function
inputs = list(range(n))
ancilla = n

# Apply the oracle gates
oracle_custom_balanced(qc_oracle_only, inputs, ancilla)

# Get the gate definition of the oracle circuit
oracle_gate = qc_oracle_only.to_gate()

# Print the gate definition (shows the gates applied)
print("Oracle Gate Definition:")
print(oracle_gate.definition)

# To get the unitary matrix, we need to simulate the circuit
# Note: Getting the unitary is only feasible for small numbers of qubits.
from qiskit_aer import Aer
from qiskit import transpile

# Create a unitary simulator backend
backend = Aer.get_backend('unitary_simulator')

# Create a circuit with just the oracle to get its unitary
qc_unitary = QuantumCircuit(n + 1)
# Apply the oracle gate
qc_unitary.append(oracle_gate, qreg)

# Run the circuit on the unitary simulator backend
job = backend.run(transpile(qc_unitary, backend))
unitary = job.result().get_unitary()

print("\nOracle Unitary Matrix:")
print(unitary)

Oracle Gate Definition:
          ┌───┐     ┌───┐┌───┐     ┌───┐                         
q_0: ──■──┤ X ├──■──┤ X ├┤ X ├──■──┤ X ├─────────────────■───────
       │  ├───┤  │  ├───┤├───┤  │  └───┘          ┌───┐  │       
q_1: ──┼──┤ X ├──■──┤ X ├┤ X ├──┼──────────────■──┤ X ├──■───────
       │  ├───┤  │  └───┘└───┘  │  ┌───┐┌───┐  │  ├───┤  │       
q_2: ──┼──┤ X ├──┼──────────────■──┤ X ├┤ X ├──■──┤ X ├──┼────■──
     ┌─┴─┐└───┘┌─┴─┐          ┌─┴─┐└───┘└───┘┌─┴─┐└───┘┌─┴─┐┌─┴─┐
q_3: ┤ X ├─────┤ X ├──────────┤ X ├──────────┤ X ├─────┤ X ├┤ X ├
     └───┘     └───┘          └───┘          └───┘     └───┘└───┘

Oracle Unitary Matrix:
Operator([[0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
           1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
           0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.